# Are We Rich Yet? - Ensemble version

## This notebook attempts to split the data into training, validation, and test in order to facilitate ensemble regression later. Training is 2009-2013, Validation is 2014, and Test is 2015.

### The predictions and ROIs are stored for each classifier run on training. A final ensemble regression is run over validation results and then applied to test results.

#### I have quite the headache.

In [1]:
%matplotlib inline

%run talibref.py

### Get data

In [2]:
#df=pd.read_csv("data/IYZ.csv")
ticker = 'IYZ'
startdate=datetime.date(2009, 1, 1)
enddate=datetime.date.today()

sdatev=datetime.date(2014, 1, 1)
edatev=datetime.date(2014, 12, 31)
sdatet=datetime.date(2015, 1, 1)
edatet=datetime.date.today()

df = generate_ticker_data(ticker, startdate, enddate)
df.head()

Available data: Ticker(IYZ) from 2000.05.26 to 2015.11.27
Usable data: Ticker(IYZ) from 2001.03.14 to 2015.11.27 
Returned data: Ticker(IYZ) from 2009.01.02 to 2015.11.27 
Save path: data/IYZ_from_2009.01.02_2015.11.27.csv


,date,open,high,low,close,volume,close_1,result_1,perf_1,close_14,result_14,perf_14,results,bb_upper,bb_middle,bb_lower,bb_pct,bb_bandwidth,bb_squeeze,bb_signalup,bb_signaldn,bb_signal,ema50,ema150,ema200,ema_signal1,ema_signal2,kama50,kama150,kama200,kama_signal1,kama_signal2,sar,sar_signal,adx,plus_di,minus_di,adx_trend,adx_direction,adx_signal,aroon_osc,aroon_signal,cci,cci_signal,macd,macd_sigline,macd_hist,macd_signal,ppo,ppo_signal,mfi,mfi_signal,roc,roc_signal,rsi,rsi_signal,ult_osc,ult_signal,willr,wr_signal,ad_osc,ad_signal,stoch_slowk,stoch_slowd,sslow_signal,stoch_fastk,stoch_fastd,srsi_signal,trix,trix_signal,sr_pivotpts,sr_res1,sr_sup1,sr_res2,sr_sup2,sr_res3,sr_sup3,cv_signal
0,2009-01-02,16.590000,17.049999,16.340000,17.000000,191500,16.799999,False,-0.011765,15.700000,False,-0.076471,0,17.014887,16.2155,15.416113,0.990688,9.859542,False,False,False,1,16.492679,19.498276,20.701668,0,0,16.286481,17.823831,17.888191,0,0,15.280000,1,9.795035,24.785122,19.663038,False,True,0,32,1,183.172819,0,0.100592,0.033176,0.067417,1,-0.005952,0,37.034034,1,3.532278,0,58.413733,1,65.340642,1,-2.824804,0,90696.048255,1,95.835932,73.235615,0,100.000000,100.000000,0,-0.318588,0,16.796666,20.943333,16.223332,21.516667,12.076665,25.663334,11.503331,0
1,2009-01-05,16.719999,16.799999,16.440001,16.799999,450700,17.129999,True,0.019643,16.030001,False,-0.045833,0,17.082515,16.2630,15.443485,0.827632,10.078272,False,False,False,1,16.504731,19.462537,20.662845,0,0,16.291986,17.800821,17.874994,0,0,15.315400,1,9.918513,23.348890,18.523617,False,True,0,28,1,123.540742,0,0.140365,0.054614,0.085752,1,-0.166423,0,36.911348,1,4.412672,0,56.286210,1,66.634350,1,-14.124302,0,282627.449178,1,90.166994,85.827476,0,79.812357,93.270786,0,-0.302988,0,16.680000,20.709999,15.989998,21.400001,11.959999,25.430000,11.269997,0
2,2009-01-06,16.980000,17.200001,16.790001,17.129999,662400,16.889999,False,-0.014011,16.299999,False,-0.048453,0,17.203151,16.3010,15.398849,0.959457,11.068655,False,False,False,1,16.529251,19.431643,20.627693,0,0,16.303562,17.786643,17.865406,0,0,15.350092,1,10.641207,26.591463,17.714263,False,True,0,20,1,169.642030,0,0.196252,0.082941,0.113311,1,-0.012625,0,45.684792,1,5.480289,0,58.943320,1,71.323459,0,-4.321108,0,474244.945173,1,91.565550,92.522825,0,100.000000,93.270786,0,-0.286544,0,17.040000,21.430001,16.710000,21.760001,12.319999,26.150002,11.989999,0
3,2009-01-07,16.790001,17.129999,16.740000,16.889999,219600,17.459999,True,0.033748,16.080000,False,-0.047957,0,17.175349,16.2940,15.412651,0.838118,10.818076,False,False,False,1,16.543398,19.397979,20.590502,0,0,16.313408,17.766881,17.852688,0,0,15.424088,1,11.202651,25.452256,17.504622,False,True,0,20,1,117.565957,0,0.218656,0.110084,0.108572,1,0.312151,1,37.529342,1,6.026359,0,56.264712,1,66.417197,0,-19.135914,0,493396.047943,1,84.298183,88.676909,0,43.076747,74.296368,0,-0.269891,0,16.919999,19.989999,16.469998,20.440000,13.399998,23.510000,12.949997,0
4,2009-01-08,16.709999,17.490000,16.709999,17.459999,464100,17.110001,False,-0.020046,16.469999,False,-0.056701,0,17.331858,16.3300,15.328142,1.063952,12.270151,False,True,False,1,16.579343,19.372310,20.559352,0,0,16.336386,17.759871,17.848124,0,0,15.495125,1,12.249067,27.201099,16.025859,False,True,0,64,1,138.281342,0,0.279187,0.143905,0.135282,1,0.393575,1,40.692428,1,10.716544,0,60.818793,1,71.626268,0,-1.570733,0,592685.130645,1,89.097527,88.320420,0,100.000000,81.025582,0,-0.252281,0,17.219999,19.619999,16.949999,19.889999,14.549999,22.289999,14.279999,0


In [3]:
df.tail()

,date,open,high,low,close,volume,close_1,result_1,perf_1,close_14,result_14,perf_14,results,bb_upper,bb_middle,bb_lower,bb_pct,bb_bandwidth,bb_squeeze,bb_signalup,bb_signaldn,bb_signal,ema50,ema150,ema200,ema_signal1,ema_signal2,kama50,kama150,kama200,kama_signal1,kama_signal2,sar,sar_signal,adx,plus_di,minus_di,adx_trend,adx_direction,adx_signal,aroon_osc,aroon_signal,cci,cci_signal,macd,macd_sigline,macd_hist,macd_signal,ppo,ppo_signal,mfi,mfi_signal,roc,roc_signal,rsi,rsi_signal,ult_osc,ult_signal,willr,wr_signal,ad_osc,ad_signal,stoch_slowk,stoch_slowd,sslow_signal,stoch_fastk,stoch_fastd,srsi_signal,trix,trix_signal,sr_pivotpts,sr_res1,sr_sup1,sr_res2,sr_sup2,sr_res3,sr_sup3,cv_signal
1734,2015-11-20,29.860001,29.900000,29.639999,29.680000,153400,29.629999,False,-0.001685,62.43750,True,1.103689,0,30.489814,29.7270,28.964186,0.469193,5.132129,False,False,False,0,29.162897,29.227979,29.312981,1,0,29.013116,29.253619,29.432630,1,0,28.890592,1,16.913323,25.572952,20.355045,False,True,1,52,1,17.310571,1,0.163693,0.224167,-0.060474,0,-0.167344,0,58.479730,0,-1.066667,0,54.495110,1,54.499672,0,-51.428571,1,385091.955069,1,88.577258,80.148257,0,72.755132,90.918377,0,0.062966,1,29.740000,31.510000,28.899999,32.350001,27.129999,34.120001,26.289998,0
1735,2015-11-23,29.690001,29.840000,29.580000,29.629999,288200,29.760000,True,0.004387,62.46875,True,1.108294,0,30.481659,29.7185,28.955341,0.442017,5.135921,False,False,False,0,29.181215,29.233304,29.316135,1,0,29.022509,29.256816,29.433652,1,0,28.951157,1,16.326191,24.382011,20.481786,False,True,1,48,1,17.048184,1,0.155769,0.210487,-0.054719,0,-0.385339,0,43.285668,0,-0.470275,0,53.636401,1,53.670164,0,-54.285771,1,284944.334474,1,77.700214,82.962588,0,49.788373,74.181168,0,0.064829,1,29.683333,31.396667,28.786666,32.293334,27.073332,34.006668,26.176665,0
1736,2015-11-24,29.549999,29.809999,29.420000,29.760000,558700,29.670000,False,-0.003024,62.00000,True,1.083333,0,30.485201,29.7370,28.988799,0.515370,5.032123,False,False,False,0,29.203912,29.240280,29.320552,1,0,29.032014,29.261284,29.435428,1,0,29.008087,1,15.281278,22.676114,21.919152,False,True,1,44,1,26.554818,1,0.158155,0.200021,-0.041866,0,-0.575353,0,41.692550,0,0.642543,0,55.595551,1,62.117478,0,-32.116838,1,350258.214409,1,73.218502,79.831991,0,77.479020,66.674175,0,0.066542,1,29.663333,31.356667,28.746666,32.273334,27.053332,33.966668,26.136665,0
1737,2015-11-25,29.770000,29.770000,29.590000,29.670000,351900,29.780001,True,0.003707,63.06250,True,1.125463,0,30.449752,29.7170,28.984248,0.467929,4.931532,False,False,False,0,29.222190,29.245972,29.324029,1,0,29.038808,29.264858,29.436786,1,0,29.061602,1,14.311001,21.914036,21.182514,False,True,1,-32,0,39.583001,1,0.151043,0.190225,-0.039182,0,-0.652351,0,48.895610,0,0.884053,0,53.897542,1,56.955565,0,-28.813559,1,332694.963982,1,64.099862,71.672859,0,10.327409,45.864934,1,0.068008,1,29.676667,31.263333,28.773333,32.166667,27.186667,33.753333,26.283333,0
1738,2015-11-27,29.650000,29.879999,29.650000,29.780001,99100,56.000000,True,0.880457,63.18750,True,1.121810,0,30.451038,29.7180,28.984962,0.542290,4.933295,False,False,False,0,29.244065,29.253045,29.328566,1,0,29.046764,29.268790,29.438804,1,0,29.111906,1,13.752790,23.059373,20.246219,False,True,1,-32,0,72.673244,1,0.152525,0.182685,-0.030160,0,-0.698883,0,51.173862,0,2.126204,0,55.679286,1,56.826208,0,-18.803333,0,299378.276190,1,68.623807,68.647390,0,100.000000,62.602143,0,0.069339,1,29.770000,30.980001,28.960000,31.790001,27.749999,33.000002,26.939999,0


In [4]:
dftouse=df.copy()

### Feature Engineering

In [5]:
IGNORE = ['date', 'result_1','close_1','perf_1','result_14','close_14','perf_14','results']

In [6]:
INDICATORS=[]
for v in df.columns:
    l=df[v].unique()
    if len(l) <= 10 and v not in IGNORE:
        print v, l
        INDICATORS.append(v)

bb_squeeze [False True]
bb_signalup [False True]
bb_signaldn [False True]
bb_signal [ 1.  0.]
ema_signal1 [0 1]
ema_signal2 [0 1]
kama_signal1 [0 1]
kama_signal2 [0 1]
sar_signal [1 0]
adx_trend [False True]
adx_direction [True False]
adx_signal [ 0.  1.]
aroon_signal [1 0]
cci_signal [ 0.  1.]
macd_signal [1 0]
ppo_signal [0 1]
mfi_signal [ 1.  0.]
roc_signal [ 0.  1.]
rsi_signal [ 1.  0.]
ult_signal [ 1.  0.]
wr_signal [ 0.  1.]
ad_signal [1 0]
sslow_signal [ 0.  1.]
srsi_signal [ 0.  1.]
trix_signal [0 1]
cv_signal [0 1]


In [7]:
STANDARDIZABLE = []
for v in df.columns:
    if v not in INDICATORS and v not in IGNORE:
        print v
        STANDARDIZABLE.append(v)

open
high
low
close
volume
bb_upper
bb_middle
bb_lower
bb_pct
bb_bandwidth
ema50
ema150
ema200
kama50
kama150
kama200
sar
adx
plus_di
minus_di
aroon_osc
cci
macd
macd_sigline
macd_hist
ppo
mfi
roc
rsi
ult_osc
willr
ad_osc
stoch_slowk
stoch_slowd
stoch_fastk
stoch_fastd
trix
sr_pivotpts
sr_res1
sr_sup1
sr_res2
sr_sup2
sr_res3
sr_sup3


In [8]:
# from sklearn.cross_validation import train_test_split
# itrain, itest = train_test_split(xrange(dftouse.shape[0]), train_size=0.7)
# mask=np.ones(dftouse.shape[0], dtype='int')
# mask[itrain]=1
# mask[itest]=0
# mask = (mask==1)
# mask.shape, mask.sum()

In [9]:
#dftouse['date'] = pd.to_datetime(dftouse['date'])
#mask = (dftouse.date < '2015-01-01').values
#mask.shape, mask.sum()

mask = dftouse['date'] < sdatev
maskv = (dftouse['date'] >= sdatev) & (dftouse['date'] <= edatev)
maskt = (dftouse['date'] >= sdatet) & (dftouse['date'] <= edatet)

### Check ROI of signals, alone:

In [10]:
#print "ROI baseline: 1.12%"
#print 'ROI "result" buy-only: 67.45%'
#print 'ROI "result" buy-sell: 172.49%'
    
def signalperf(signal, valtest):
    ypred = df[signal]
    if valtest == "v":
        df_pred = df[maskv].reset_index(drop=True)
        sdate = sdatev
        edate = edatev
    else:
        df_pred = df[maskt].reset_index(drop=True)
        sdate = sdatet
        edate = edatet
    df_pred['pred_result'] = ypred
    df_pred['result_baseline'] = np.ones(df_pred.shape[0])

    balance, profit, ROI2, balovertime, signals = evaluate_profit(df_pred, sdate, edate, 10000, 'pred_result', 'close', False, [0])
    #print 'ROI "pred" buy-only: {0:.2f}%'.format(ROI*100)

    balance, profit, ROI, balovertime, signals = evaluate_profit(df_pred, sdate, edate, 10000, 'pred_result', 'close', False, [1])
    print signal + ': ROI "pred" buy-only: {0:.2f}%'.format(ROI2*100), 'buy-sell: {0:.2f}%'.format(ROI*100)
    return

In [11]:
print 'validation: ' + sdatev.strftime('%Y-%m-%d') + ' - ' + edatev.strftime('%Y-%m-%d')

for v in INDICATORS:
    signalperf(v, "v")

validation: 2014-01-01 - 2014-12-31
bb_squeeze: ROI "pred" buy-only: 1.03% buy-sell: 0.80%
bb_signalup: ROI "pred" buy-only: -3.14% buy-sell: -3.26%
bb_signaldn: ROI "pred" buy-only: 5.02% buy-sell: 5.05%
bb_signal: ROI "pred" buy-only: 3.94% buy-sell: 3.59%
ema_signal1: ROI "pred" buy-only: -5.52% buy-sell: -5.85%
ema_signal2: ROI "pred" buy-only: -5.05% buy-sell: -5.26%
kama_signal1: ROI "pred" buy-only: -1.69% buy-sell: -2.01%
kama_signal2: ROI "pred" buy-only: -1.76% buy-sell: -2.20%
sar_signal: ROI "pred" buy-only: -6.28% buy-sell: -6.60%
adx_trend: ROI "pred" buy-only: -6.23% buy-sell: -6.28%
adx_direction: ROI "pred" buy-only: -12.19% buy-sell: -11.85%
adx_signal: ROI "pred" buy-only: -0.10% buy-sell: -0.57%
aroon_signal: ROI "pred" buy-only: -7.76% buy-sell: -7.71%
cci_signal: ROI "pred" buy-only: -0.98% buy-sell: -1.41%
macd_signal: ROI "pred" buy-only: -5.03% buy-sell: -5.21%
ppo_signal: ROI "pred" buy-only: -11.77% buy-sell: -11.46%
mfi_signal: ROI "pred" buy-only: 5.53% buy

In [12]:
print 'test: ' + sdatet.strftime('%Y-%m-%d') + '-' + edatet.strftime('%Y-%m-%d')

for v in INDICATORS:
    signalperf(v, "t")

test: 2015-01-01-2015-11-28
bb_squeeze: ROI "pred" buy-only: 0.74% buy-sell: 3.26%
bb_signalup: ROI "pred" buy-only: -7.86% buy-sell: -5.24%
bb_signaldn: ROI "pred" buy-only: -7.18% buy-sell: -5.12%
bb_signal: ROI "pred" buy-only: -1.98% buy-sell: -0.51%
ema_signal1: ROI "pred" buy-only: -1.58% buy-sell: 0.48%
ema_signal2: ROI "pred" buy-only: 3.13% buy-sell: 4.74%
kama_signal1: ROI "pred" buy-only: -6.18% buy-sell: -4.26%
kama_signal2: ROI "pred" buy-only: -2.35% buy-sell: -0.86%
sar_signal: ROI "pred" buy-only: -11.73% buy-sell: -8.27%
adx_trend: ROI "pred" buy-only: -9.68% buy-sell: -7.25%
adx_direction: ROI "pred" buy-only: -1.60% buy-sell: 0.68%
adx_signal: ROI "pred" buy-only: -11.56% buy-sell: -9.24%
aroon_signal: ROI "pred" buy-only: -5.93% buy-sell: -3.82%
cci_signal: ROI "pred" buy-only: 3.12% buy-sell: 5.49%
macd_signal: ROI "pred" buy-only: -9.77% buy-sell: -6.64%
ppo_signal: ROI "pred" buy-only: -1.76% buy-sell: 0.94%
mfi_signal: ROI "pred" buy-only: -0.46% buy-sell: 1.30%

#### 1.2 Standardize the data

Use the mask to compute the training and test parts of the dataframe. Use `StandardScaler` from `sklearn.preprocessing` to "fit" the columns in `STANDARDIZABLE` on the training set. Then use the resultant estimator to transform both the training and the test parts of each of the columns in the dataframe, replacing the old unstandardized values in the `STANDARDIZABLE` columns of `dftouse` by the new standardized ones.

In [13]:
#your code here
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(dftouse[mask][STANDARDIZABLE])
dftouse[STANDARDIZABLE] = scaler.transform(dftouse[STANDARDIZABLE])
dftouse.head()

,date,open,high,low,close,volume,close_1,result_1,perf_1,close_14,result_14,perf_14,results,bb_upper,bb_middle,bb_lower,bb_pct,bb_bandwidth,bb_squeeze,bb_signalup,bb_signaldn,bb_signal,ema50,ema150,ema200,ema_signal1,ema_signal2,kama50,kama150,kama200,kama_signal1,kama_signal2,sar,sar_signal,adx,plus_di,minus_di,adx_trend,adx_direction,adx_signal,aroon_osc,aroon_signal,cci,cci_signal,macd,macd_sigline,macd_hist,macd_signal,ppo,ppo_signal,mfi,mfi_signal,roc,roc_signal,rsi,rsi_signal,ult_osc,ult_signal,willr,wr_signal,ad_osc,ad_signal,stoch_slowk,stoch_slowd,sslow_signal,stoch_fastk,stoch_fastd,srsi_signal,trix,trix_signal,sr_pivotpts,sr_res1,sr_sup1,sr_res2,sr_sup2,sr_res3,sr_sup3,cv_signal
0,2009-01-02,-1.621906,-1.535739,-1.633611,-1.497979,-0.755747,16.799999,False,-0.011765,15.700000,False,-0.076471,0,-1.729910,-1.723647,-1.696349,1.291473,0.570058,False,False,False,1,-1.644269,-0.747476,-0.336152,0,0,-1.702461,-1.206145,-1.205757,0,0,-1.942683,1,-1.784421,0.017917,-0.469699,False,True,0,0.272443,1,1.546019,0,0.121883,-0.159758,0.776891,1,-0.182142,0,-0.929925,1,0.817655,0,0.437435,1,1.073629,1,1.237504,0,-0.086803,1,1.501523,0.690471,0,1.146637,1.442468,0,-3.515149,0,-1.556557,-0.701654,-1.383920,-0.812185,-2.168072,0.015499,-1.937205,0
1,2009-01-05,-1.583502,-1.609924,-1.604310,-1.556946,-0.098644,17.129999,True,0.019643,16.030001,False,-0.045833,0,-1.709687,-1.709451,-1.688274,0.804131,0.626965,False,False,False,1,-1.640565,-0.760233,-0.351401,0,0,-1.700785,-1.213439,-1.209976,0,0,-1.932369,1,-1.769049,-0.184816,-0.613639,False,True,0,0.208667,1,0.996396,0,0.281656,-0.066339,0.989488,1,-0.273169,0,-0.936891,1,1.058551,0,0.248499,1,1.196477,1,0.874819,0,0.391447,1,1.285511,1.207257,0,0.668872,1.242026,0,-3.362222,0,-1.590969,-0.767787,-1.448406,-0.846701,-2.200462,-0.048689,-1.997159,0
2,2009-01-06,-1.506693,-1.491228,-1.501758,-1.459651,0.438041,16.889999,False,-0.014011,16.299999,False,-0.048453,0,-1.673613,-1.698095,-1.701442,1.198129,0.884631,False,False,False,1,-1.633030,-0.771261,-0.365209,0,0,-1.697261,-1.217934,-1.213041,0,0,-1.922261,1,-1.679082,0.272893,-0.715882,False,True,0,0.081114,1,1.421307,0,0.506158,0.057104,1.309032,1,-0.185928,0,-0.438754,1,1.350675,0,0.484465,1,1.641744,0,1.189476,0,0.868914,1,1.338802,1.482043,0,1.146637,1.242026,0,-3.201022,0,-1.484783,-0.563717,-1.249420,-0.740194,-2.100514,0.149377,-1.812159,0
3,2009-01-07,-1.562822,-1.512000,-1.516409,-1.530411,-0.684510,17.459999,True,0.033748,16.080000,False,-0.047957,0,-1.681927,-1.700187,-1.697370,0.835470,0.819438,False,False,False,1,-1.628683,-0.783278,-0.379817,0,0,-1.694263,-1.224198,-1.217106,0,0,-1.900702,1,-1.609189,0.112087,-0.742365,False,True,0,0.081114,1,0.941327,0,0.596157,0.175384,1.254084,1,-0.001700,1,-0.901803,1,1.500092,0,0.246590,1,1.175857,0,0.713959,0,0.916635,1,1.061883,1.324202,0,-0.200521,0.676837,0,-3.037773,0,-1.520179,-0.971858,-1.315749,-1.130717,-1.800669,-0.576865,-1.565493,0
4,2009-01-08,-1.586456,-1.405174,-1.525199,-1.362357,-0.064673,17.110001,False,-0.020046,16.469999,False,-0.056701,0,-1.635126,-1.689429,-1.722301,1.510443,1.197223,False,True,False,1,-1.617637,-0.792440,-0.392052,0,0,-1.687268,-1.226420,-1.218565,0,0,-1.880005,1,-1.478921,0.358947,-0.929173,False,True,0,0.782652,1,1.132259,0,0.839318,0.322762,1.563794,1,0.044488,1,-0.722210,1,2.783433,0,0.651017,1,1.670498,0,1.277756,0,1.164041,1,1.244759,1.309571,0,1.146637,0.877279,0,-2.865153,0,-1.431691,-1.076727,-1.183091,-1.293434,-1.481389,-0.912476,-1.223757,0


We create a list `lcols` of the columns we will use in our classifier. This list should not contain the response `RESP`. How many features do we have?

In [14]:
#lcols=list(dftouse.columns)
#lcols.remove(u'results')
lcols=[]
#lcols.append('cv_signal')
for c in list(dftouse.columns):
    #if c not in INDICATORS and c not in IGNORE:
    #if c in INDICATORS:
    if c not in IGNORE:  #Original
        lcols.append(c)
print len(lcols)

70


### EDA for the data

We create a variable `ccols` which contains all variables not in our indicators list

In [15]:
ccols=[]
for c in lcols:
    if c not in INDICATORS and c not in IGNORE:
        ccols.append(c)
print len(ccols), len(INDICATORS)
ccols

44 26


['open',
 'high',
 'low',
 'close',
 'volume',
 'bb_upper',
 'bb_middle',
 'bb_lower',
 'bb_pct',
 'bb_bandwidth',
 'ema50',
 'ema150',
 'ema200',
 'kama50',
 'kama150',
 'kama200',
 'sar',
 'adx',
 'plus_di',
 'minus_di',
 'aroon_osc',
 'cci',
 'macd',
 'macd_sigline',
 'macd_hist',
 'ppo',
 'mfi',
 'roc',
 'rsi',
 'ult_osc',
 'willr',
 'ad_osc',
 'stoch_slowk',
 'stoch_slowd',
 'stoch_fastk',
 'stoch_fastd',
 'trix',
 'sr_pivotpts',
 'sr_res1',
 'sr_sup1',
 'sr_res2',
 'sr_sup2',
 'sr_res3',
 'sr_sup3']

#### 1.4 Train a SVM on this data.

In [16]:
from sklearn.svm import LinearSVC
from sklearn.grid_search import GridSearchCV

In [17]:
"""
Function
--------
cv_optimize

Inputs
------
clf : an instance of a scikit-learn classifier
parameters: a parameter grid dictionary thats passed to GridSearchCV (see above)
X: a samples-features matrix in the scikit-learn style
y: the response vectors of 1s and 0s (+ives and -ives)
n_folds: the number of cross-validation folds (default 5)
score_func: a score function we might want to pass (default python None)
   
Returns
-------
The best estimator from the GridSearchCV, after the GridSearchCV has been used to
fit the model.
     
Notes
-----
see do_classify and the code below for an example of how this is used
"""
#your code here
def cv_optimize(clf, parameters, X, y, n_folds, score_func):
    fitmodel = GridSearchCV(clf, param_grid=parameters, cv=n_folds, scoring=score_func)
    fitmodel.fit(X, y)
    return fitmodel.best_estimator_

In [18]:
from sklearn.metrics import confusion_matrix
def do_classify(clf, parameters, indf, featurenames, targetname, target1val, mask=mask, reuse_split=None, score_func=None, n_folds=5):
    subdf=indf[featurenames]
    X=subdf#.values
    y=(indf[targetname].values==target1val)*1
    y=indf[targetname]
    #if mask !=None:
    #    print "using mask"
    #    Xtrain, Xval, Xtest, ytrain, yval, ytest = X[mask], X[maskv], X[maskt], y[mask], y[maskv], y[maskt]
    Xtrain, Xval, Xtest, ytrain, yval, ytest = X[mask], X[maskv], X[maskt], y[mask], y[maskv], y[maskt]
    if reuse_split !=None:
        print "using reuse split"
        Xtrain, Xval, Xtest, ytrain, yval, ytest = reuse_split['Xtrain'], reuse_split['Xval'], reuse_split['Xtest'], reuse_split['ytrain'], reuse_split['yval'], reuse_split['ytest']
    if parameters:
        clf = cv_optimize(clf, parameters, Xtrain, ytrain, n_folds=n_folds, score_func=score_func)
    clf=clf.fit(Xtrain, ytrain)
    training_accuracy = clf.score(Xtrain, ytrain)
    val_accuracy = clf.score(Xval, yval)
    test_accuracy = clf.score(Xtest, ytest)
    print "############# based on standard predict ################"
    print "Accuracy on training data: %0.2f" % (training_accuracy)
    print "Accuracy on validation data: %0.2f" % (val_accuracy)
    print "Accuracy on test data:     %0.2f" % (test_accuracy)
    print confusion_matrix(ytest, clf.predict(Xtest))
    print "########################################################"
    return clf, Xtrain, ytrain, Xval, yval, Xtest, ytest

In [19]:
%%time
clfsvm, Xtrain, ytrain, Xval, yval, Xtest, ytest = do_classify(LinearSVC(loss="hinge"), {"C": [0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0, 10.0]}, dftouse, lcols, u'results', 1, mask=mask)

############# based on standard predict ################
Accuracy on training data: 0.69
Accuracy on validation data: 0.73
Accuracy on test data:     0.72
[[89 36]
 [29 75]]
########################################################
CPU times: user 1.76 s, sys: 14 ms, total: 1.77 s
Wall time: 1.87 s


In [20]:
clfsvm

LinearSVC(C=0.01, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0)

In [21]:
predsv={}
predsv_ROI_long={}
predsv_ROI_longshort={}

predst={}
predst_ROI_long={}
predst_ROI_longshort={}

In [22]:
df_pred = df[maskv].reset_index(drop=True)
df_pred['baseline'] = np.ones(df_pred.shape[0])

balance, profit, ROI, balovertime, signals = evaluate_profit(df_pred, sdatev, edatev, 10000, 'baseline', 'close', False, [0])
print "ROI baseline: {0:.2f}%".format(ROI*100)
predsv["baseline_long"] = df_pred['baseline']
predsv_ROI_long["baseline_long"] = ROI
predsv_ROI_longshort["baseline_long"] = ROI

balance, profit, ROI, balovertime, signals = evaluate_profit(df_pred, sdatev, edatev, 10000, 'results', 'close', False, [0])
print 'ROI "result" buy-only: {0:.2f}%'.format(ROI*100)
predsv["baseline_ema"] = df_pred['results']
predsv_ROI_long["baseline_ema"] = ROI
balance, profit, ROI, balovertime, signals = evaluate_profit(df_pred, sdatev, edatev, 10000, 'results', 'close', False, [1])
print 'ROI "result" buy-sell: {0:.2f}%'.format(ROI*100)
predsv_ROI_longshort["baseline_ema"] = ROI

balance, profit, ROI, balovertime, signals = evaluate_profit(df_pred, sdatev, edatev, 10000, 'result_1', 'close', False, [0])
print 'ROI "result" buy-only: {0:.2f}%'.format(ROI*100)
predsv["baseline_max"] = df_pred['result_1']
predsv_ROI_long["baseline_max"] = ROI
balance, profit, ROI, balovertime, signals = evaluate_profit(df_pred, sdatev, edatev, 10000, 'result_1', 'close', False, [1])
print 'ROI "result" buy-sell: {0:.2f}%'.format(ROI*100)
predsv_ROI_longshort["baseline_max"] = ROI

ROI baseline: 0.00%
ROI "result" buy-only: 44.51%
ROI "result" buy-sell: 45.61%
ROI "result" buy-only: 133.45%
ROI "result" buy-sell: 137.70%


In [23]:
df_pred = df[maskt].reset_index(drop=True)
df_pred['baseline'] = np.ones(df_pred.shape[0])

balance, profit, ROI, balovertime, signals = evaluate_profit(df_pred, sdatet, edatet, 10000, 'baseline', 'close', False, [0])
print "ROI baseline: {0:.2f}%".format(ROI*100)
predst["baseline_long"] = df_pred['baseline']
predst_ROI_long["baseline_long"] = ROI
predst_ROI_longshort["baseline_long"] = ROI

balance, profit, ROI, balovertime, signals = evaluate_profit(df_pred, sdatet, edatet, 10000, 'results', 'close', False, [0])
print 'ROI "result" buy-only: {0:.2f}%'.format(ROI*100)
predst["baseline_ema"] = df_pred['results']
predst_ROI_long["baseline_ema"] = ROI
balance, profit, ROI, balovertime, signals = evaluate_profit(df_pred, sdatet, edatet, 10000, 'results', 'close', False, [1])
print 'ROI "result" buy-sell: {0:.2f}%'.format(ROI*100)
predst_ROI_longshort["baseline_ema"] = ROI

balance, profit, ROI, balovertime, signals = evaluate_profit(df_pred, sdatet, edatet, 10000, 'result_1', 'close', False, [0])
print 'ROI "result" buy-only: {0:.2f}%'.format(ROI*100)
predst["baseline_max"] = df_pred['result_1']
predst_ROI_long["baseline_max"] = ROI
balance, profit, ROI, balovertime, signals = evaluate_profit(df_pred, sdatet, edatet, 10000, 'result_1', 'close', False, [1])
print 'ROI "result" buy-sell: {0:.2f}%'.format(ROI*100)
predst_ROI_longshort["baseline_max"] = ROI

ROI baseline: 0.00%
ROI "result" buy-only: 62.68%
ROI "result" buy-sell: 68.01%
ROI "result" buy-only: 153.57%
ROI "result" buy-sell: 166.25%


In [24]:
def evaluate(clf, desc):

    #Validation
    ypred = clf.predict(Xval)
    df_pred = df[maskv].reset_index(drop=True)
    df_pred['pred_result'] = ypred
    df_pred['result_baseline'] = np.ones(df_pred.shape[0])
    print "accuracy on validation set: {0:.3f}".format((df_pred.result_14 == df_pred.pred_result).sum()/float(len(df_pred)))
    
    balance, profit, ROI, balovertime, signals = evaluate_profit(df_pred, startdate, enddate, 10000, 'pred_result', 'close', False, [0])
    print 'ROI "pred" buy-only: {0:.2f}%'.format(ROI*100)
    predsv[desc] = ypred
    predsv_ROI_long[desc] = ROI

    balance, profit, ROI, balovertime, signals = evaluate_profit(df_pred, startdate, enddate, 10000, 'pred_result', 'close', False, [1])
    print 'ROI "pred" buy-sell: {0:.2f}%'.format(ROI*100)
    predsv_ROI_longshort[desc] = ROI
    
    #Test
    ypred = clf.predict(Xtest)
    df_pred = df[maskt].reset_index(drop=True)
    df_pred['pred_result'] = ypred
    df_pred['result_baseline'] = np.ones(df_pred.shape[0])
    print "accuracy on test set: {0:.3f}".format((df_pred.result_14 == df_pred.pred_result).sum()/float(len(df_pred)))
    
    balance, profit, ROI, balovertime, signals = evaluate_profit(df_pred, startdate, enddate, 10000, 'pred_result', 'close', False, [0])
    print 'ROI "pred" buy-only: {0:.2f}%'.format(ROI*100)
    predst[desc] = ypred
    predst_ROI_long[desc] = ROI

    balance, profit, ROI, balovertime, signals = evaluate_profit(df_pred, startdate, enddate, 10000, 'pred_result', 'close', False, [1])
    print 'ROI "pred" buy-sell: {0:.2f}%'.format(ROI*100)
    predst_ROI_longshort[desc] = ROI

In [25]:
evaluate(clfsvm, "svm")

accuracy on validation set: 0.575
ROI "pred" buy-only: 6.93%
ROI "pred" buy-sell: 6.82%
accuracy on test set: 0.568
ROI "pred" buy-only: 5.44%
ROI "pred" buy-sell: 7.43%


The results obtained ought to be very similar to the efforts you put in earlier. If not its likely you wrote `cv_optimize` wrong. (Remember that we are using the same mask).

We'll reuse the training and test sets you computed above later in the homework. We do this by putting them into a dictionary `reuse_split`

In [26]:
reuse_split=dict(Xtrain=Xtrain, Xval=Xval, Xtest=Xtest, ytrain=ytrain, yval=yval, ytest=ytest)

## 2. Estimate costs and benefits from assumptions and data

### Our data is highly asymmetric

First notice that our data set is very highly asymmetric, with positive `RESP`onses only making up 16-17% of the samples.

In [27]:
print "whole data set", dftouse['results'].mean()#Highly asymmetric
print "training set", dftouse['results'][mask].mean(), "val set", dftouse['results'][maskv].mean(), "test set", dftouse['results'][maskt].mean()

whole data set 0.470385278896
training set 0.476152623211 val set 0.456349206349 test set 0.454148471616


This means that a classifier which predicts that EVERY customer is a negative has an accuracy rate of 83-84%. By this we mean that **a classifier that predicts that no customer will respond to our mailing** has an accuracy of 83-84%!

In [28]:
#your code here
from sklearn.linear_model import LogisticRegression
clflog = LogisticRegression(penalty="l1")
parameters = {"C": [0.001, 0.01, 0.1, 1, 10, 100]}
clflog, Xtrain, ytrain, Xval, yval, Xtest, ytest = do_classify(clflog, parameters, dftouse, lcols, u'results',1, mask=mask)#, reuse_split=reuse_split)

############# based on standard predict ################
Accuracy on training data: 0.66
Accuracy on validation data: 0.72
Accuracy on test data:     0.69
[[78 47]
 [23 81]]
########################################################


In [29]:
clflog

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [30]:
evaluate(clflog, "logistic_regression")

accuracy on validation set: 0.587
ROI "pred" buy-only: 9.09%
ROI "pred" buy-sell: 8.98%
accuracy on test set: 0.546
ROI "pred" buy-only: 8.34%
ROI "pred" buy-sell: 10.47%


In [31]:
from sklearn.ensemble import RandomForestClassifier
clfraf = RandomForestClassifier(n_jobs=2)
parameters = {}#{"C": [0.001, 0.01, 0.1, 1, 10, 100]}
clfraf, Xtrain, ytrain, Xval, yval, Xtest, ytest = do_classify(clfraf, parameters, dftouse, lcols, u'results',1, mask=mask, reuse_split=reuse_split)

using reuse split
############# based on standard predict ################
Accuracy on training data: 0.99
Accuracy on validation data: 0.66
Accuracy on test data:     0.56
[[72 53]
 [47 57]]
########################################################


In [32]:
evaluate(clfraf, "random_forest")

accuracy on validation set: 0.607
ROI "pred" buy-only: 11.31%
ROI "pred" buy-sell: 11.72%
accuracy on test set: 0.528
ROI "pred" buy-only: -1.91%
ROI "pred" buy-sell: 0.37%


## 4. Trying to improve the SVM: Feature Selection and Data Balancing

If you did the previous section right, you will find that the logistic regression model provides a better profit over some section of the profit curve than the baseline "send to everyone" classifier, while the SVM classifier is generally poor. At this might we might want to try all kinds of classifiers: from perceptrons to random forests. In the interest of time, and to study the SVM in some more detail, we'll restrict ourselves to trying to improve the SVM performance here. In real life you would try other classifiers as well.

 We wont be exhaustive in this improvement process either(which is something you should do on your project) in the interests of time, but we'll explore if feature-selection on the  SVM, and data balancing on the SVM (SVM's are known to perform better on balanced data) help.
 
( An aside: many classifiers such as SVM and decision trees struggle in their techniques on imbalanced data. You can read more at: see Weiss, Gary M., and Foster Provost. "The effect of class distribution on classifier learning: an empirical study." Rutgers Univ (2001). Also see http://pages.stern.nyu.edu/~fprovost/Papers/skew.PDF and http://www.cs.ox.ac.uk/people/vasile.palade/papers/Class-Imbalance-SVM.pdf for multiple ways to deal with the imbalance problem: balancing is not always the best option. `Sklearn` also provides a class weighting strategy: http://scikit-learn.org/stable/modules/svm.html#unbalanced-problems ). 

### Feature Selection

The Lasso, for example, implements internally, a form of feature selection by setting many coefficients to zero. Let us find coefficients that are non-zero.

#### Non zero lasso features

We write a function `nonzero_lasso` which takes the fit classifier `clfloglasso` as an argument, and spits out a dataframe of coefficients, sorted by the absolute magnitude of the coefficients. This way we can see which features dominated the logistic regression.

In [33]:
def nonzero_lasso(clf):
    featuremask=(clf.coef_ !=0.0)[0]
    return pd.DataFrame(dict(feature=lcols, coef=clf.coef_[0], abscoef=np.abs(clf.coef_[0])))[featuremask].sort('abscoef', ascending=False)

In [34]:
lasso_importances=nonzero_lasso(clflog)
lasso_importances.set_index("feature", inplace=True)
lasso_importances.head(10)

,abscoef,coef
feature,,
stoch_slowk,0.252737,-0.252737
rsi,0.209237,-0.209237


#### 4.1 Feature importance using correlations

We can also get a notion of which features are important in the classification process by seeing how they correlate with the response. Implement some code to obtain the Pearson correlation coefficient between each of our features and the response. Do this on the training set only! Create a dataframe indexed by the features, which has columns `abscorr` the absolute value of the correlation and `corr` the value of the correlation. Sort the dataframe by `abscorr`, highest first, and show the top 25 features with the highest absolute correlation. Is there much overlap with the feature selection performed by the LASSO?

In [35]:
from scipy.stats.stats import pearsonr
correlations=[]
dftousetrain=dftouse[mask]
for col in lcols:
    r=pearsonr(dftousetrain[col], dftousetrain['results'])[0]
    correlations.append(dict(feature=col,corr=r, abscorr=np.abs(r)))

bpdf=pd.DataFrame(correlations).sort('abscorr', ascending=False)
bpdf.set_index(['feature'], inplace=True)
bpdf.head(25)

,abscorr,corr
feature,,
stoch_slowk,0.349108,-0.349108
rsi,0.341184,-0.341184
cci,0.332055,-0.332055
bb_pct,0.320266,-0.320266
willr,0.312022,-0.312022
stoch_slowd,0.310636,-0.310636
plus_di,0.309800,-0.309800
ult_osc,0.304699,-0.304699
stoch_fastd,0.294346,-0.294346


#### Why Feature Select?

One of the reasons feature selection is done, automatically or otherwise, is that there might be strong correlations between features. Also recall polynomial regression: a large number of features can lead to overfitting. Feature selection helps curb the problem of the curse of dimensionality, where centrality measures often used in statistics go wonky at higher dimensions. Between feature-engineering which we did some of, earlier, and feature selection, is where a lot of smarts and domain knowledge comes in. You will gain this with experience.

### Create a pipeline to feature-select, standardize and train!

We shall use sklearn pipelines to do correlation-with-response based feature selection for our SVM model. Maybe such feature-selection will improve the abysmal performance. 

This does not reduce the collinearity amongst the features, for which one either needs PCA, ICA, or some feature selection using the forward-backward algorithm. We do not have the time to approach it here. 

Its very important to do response based feature selection in the right way. If you remember, we separately standardized the training and test sets. This was to prevent **any** information about the overall mean and standard deviation leaking into the test set. 

But we played a bit loose with the rules there. We standardized on the entire training set. Instead we should have been standardizing separately in each cross-validation fold. There the original training set would be broken up into a sub-training and validation set, the standardization needed to be done on those separately. This can be implemented with `sklearn` pipelines.

Such kind of "data snooping" is relatively benign though, as it used no information about the response variable. But if you do any feature selection which uses the response variable, such as choosing the "k" most correlated variables from above, its not benign any more. This is because you have leaked the response from the validation into your sub-training set, and cannot thus be confident about your predictions: you might overfit. In such a situation, you must do the feature selection inside the cross-validation fold. See http://nbviewer.ipython.org/github/cs109/content/blob/master/lec_10_cross_val.ipynb from the 2013 course for a particularly dastardly case of this, where you see that the problem is particularly exacerbated when you have many more features than samples.

Lets do this here using sklearn pipelines.

In [36]:
from sklearn import feature_selection
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest

Lets define a scorer which returns the absolute values of the pearson correlation between the feature and the response for each sample. The specific form of the scorer is dictated to us in the API docs for `SelectKBest`, see [here](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html): the first argument must be an array of scores, and the second an array of p-values.

In [37]:
def pearson_scorer(X,y):
    rs=np.zeros(X.shape[1])
    pvals=np.zeros(X.shape[1])
    i=0
    for v in X.T:
        rs[i], pvals[i]=pearsonr(v, y)
        i=i+1
    return np.abs(rs), pvals    

Lets apply the feature selection to a model which did not have any automatic feature selection and performed rather poorly before: the linear SVM. 

The `Pipeline` feature of sklearn chains various parts of a machine learning algorithm together. In this case we want to chain feature-selection and training in such a way that both happen freshly for each cross-validation fold (we wont bother to standardize in each cross-validation fold separately here for brevity, although you might want to do this).
We use the `SelectKBest` meta estimator to select the 25 most correlated/anti-correlated features. We create an instance of this meta-estimator, `selectorlinearsvm`. We then combine it with the linear SVC estimators into the pipeline `pipelinearsvm`: the `Pipeline` function simply takes a list of `scikit-learn` estimators and wraps them together into a new estimator object, which can then be passed to `GridSearchCV` via our `do_classify` function. Notice how this new estimator object can be used exactly the same way as a single classifier can be used in `scikit-learn`..this uniformity of interface is one of the nice features of `sklearn`!

In [38]:
selectorlinearsvm = SelectKBest(k=25, score_func=pearson_scorer)
pipelinearsvm = Pipeline([('select', selectorlinearsvm), ('svm', LinearSVC(loss="hinge"))])


#### Let us run the pipelined classifier 

We'll run the classifier and compare the results using the ROC curve to the previous SVM result.

In [39]:
pipelinearsvm, _,_,_,_,_,_  = do_classify(pipelinearsvm, {"svm__C": [0.00001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}, dftouse,lcols, u'results',1, reuse_split=reuse_split)

using reuse split
############# based on standard predict ################
Accuracy on training data: 0.69
Accuracy on validation data: 0.71
Accuracy on test data:     0.72
[[91 34]
 [29 75]]
########################################################


What features did the pipelined classifier use? We can access them so:

In [40]:
np.array(lcols)[pipelinearsvm.get_params()['select'].get_support()]

array(['bb_pct', 'ema_signal1', 'kama_signal1', 'sar_signal', 'plus_di',
       'minus_di', 'adx_direction', 'aroon_osc', 'cci', 'macd',
       'macd_hist', 'macd_signal', 'mfi', 'roc', 'rsi', 'ult_osc', 'willr',
       'wr_signal', 'ad_osc', 'stoch_slowk', 'stoch_slowd', 'sslow_signal',
       'stoch_fastk', 'stoch_fastd', 'srsi_signal'], 
      dtype='|S13')

We plot the ROC curves, using the label `svm-feature-selected` for the pipelined classifier `pipelinearsvm`. We plot it alongside the older logistic-with lasso and all-features SVM for comparison

In [41]:
evaluate(pipelinearsvm, "pipelinearsvm")

accuracy on validation set: 0.560
ROI "pred" buy-only: 5.01%
ROI "pred" buy-sell: 4.92%
accuracy on test set: 0.559
ROI "pred" buy-only: 6.54%
ROI "pred" buy-sell: 8.67%


#### 4.3 Implement a RBF based pipelined (feature-selected) classifier on the balanced set.

In [42]:
from sklearn.svm import SVC
reuse_split_new = reuse_split

In [43]:
%%time
selectorsvm2 = SelectKBest(k=25, score_func=pearson_scorer)
pipesvm2 = Pipeline([('select2', selectorsvm2), ('svm2', SVC())])
pipesvm2, _,_,_,_,_,_  = do_classify(pipesvm2, {"svm2__C": [1e8, 1e9, 1e10], "svm2__gamma": [ 1e-9, 1e-10, 1e-11]}, dftouse,lcols, u'results',1, reuse_split=reuse_split)

using reuse split
############# based on standard predict ################
Accuracy on training data: 0.67
Accuracy on validation data: 0.73
Accuracy on test data:     0.71
[[90 35]
 [32 72]]
########################################################
CPU times: user 4.25 s, sys: 18.8 ms, total: 4.27 s
Wall time: 4.31 s


In [44]:
pipesvm2.get_params()

{'select2': SelectKBest(k=25, score_func=<function pearson_scorer at 0x109b24398>),
 'select2__k': 25,
 'select2__score_func': <function __main__.pearson_scorer>,
 'svm2': SVC(C=100000000.0, cache_size=200, class_weight=None, coef0=0.0, degree=3,
   gamma=1e-09, kernel='rbf', max_iter=-1, probability=False,
   random_state=None, shrinking=True, tol=0.001, verbose=False),
 'svm2__C': 100000000.0,
 'svm2__cache_size': 200,
 'svm2__class_weight': None,
 'svm2__coef0': 0.0,
 'svm2__degree': 3,
 'svm2__gamma': 1e-09,
 'svm2__kernel': 'rbf',
 'svm2__max_iter': -1,
 'svm2__probability': False,
 'svm2__random_state': None,
 'svm2__shrinking': True,
 'svm2__tol': 0.001,
 'svm2__verbose': False}

In [45]:
evaluate(pipesvm2, "pipesvm2")

accuracy on validation set: 0.595
ROI "pred" buy-only: 6.43%
ROI "pred" buy-sell: 6.22%
accuracy on test set: 0.576
ROI "pred" buy-only: 15.31%
ROI "pred" buy-sell: 17.45%


## Yvan ML

In [46]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import f1_score

# selector of  features
featSelector = SelectKBest(k=18, score_func=pearson_scorer)

print "#############====================== Log Regression =====================#############"
pipeLR = Pipeline([('select', featSelector), ('LR', LogisticRegression(penalty="l1"))])
pipeLR, _,_,_,_,_,_  = do_classify(pipeLR, {"LR__C": [0.005, 0.01, 0.02, 0.05, 10.0]}, dftouse,lcols, u'results',1, reuse_split=reuse_split_new)
evaluate(pipeLR, "pipe_Log_Regr")
print "#############====================== Linear SVM ========================#############"
clfsvm_b = Pipeline([('select', featSelector), ('svm', LinearSVC(loss="hinge"))])
clfsvm_b, _,_,_,_,_,_  = do_classify(clfsvm_b, {"svm__C": [0.00001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}, dftouse,lcols, u'results',1, reuse_split=reuse_split_new)
evaluate(clfsvm_b, "linear_svm")
print "#############====================== RBF SVC ===========================#############"
pipesvm2 = Pipeline([('select', featSelector), ('svm2', SVC())])
pipesvm2, _,_,_,_,_,_  = do_classify(pipesvm2, {"svm2__C": [1e8, 1e9, 1e10], "svm2__gamma": [ 1e-9, 1e-10, 1e-11]}, dftouse,lcols, u'results',1, reuse_split=reuse_split_new)
evaluate(pipesvm2, "rbf_svc")
print "#############====================== Random Forest =====================#############"
pipeRF = Pipeline([('select', featSelector), ('RF', RandomForestClassifier())])
pipeRF, _,_,_,_,_,_  = do_classify(pipeRF, {"RF__max_depth": [3,5,7,10,15,25,50], "RF__n_estimators": [5,10,20,40],"RF__max_features": [1,2,3]}, dftouse,lcols, u'results',1, reuse_split=reuse_split_new)
evaluate(pipeRF, "rand_forest")
print "#############====================== Extra Trees= =====================#############"
pipeET = Pipeline([('select', featSelector), ('ET', RandomForestClassifier())])
pipeET, _,_,_,_,_,_  = do_classify(pipeET, {"ET__max_depth": [3,5,7,10,15,25,50], "ET__n_estimators": [5,10,20,40],"ET__max_features": [1,2,3]}, dftouse,lcols, u'results',1, reuse_split=reuse_split_new)
evaluate(pipeET, "extra_trees")
print "#############====================== AdaBoost ==========================#############"
pipeAda = Pipeline([('select', featSelector), ('Ada', AdaBoostClassifier())])
pipeAda, _,_,_,_,_,_  = do_classify(pipeAda, {"Ada__n_estimators": [5,10,20,40],"Ada__learning_rate": [0.1,0.5,1.0]}, dftouse,lcols, u'results',1, reuse_split=reuse_split_new)
evaluate(pipeAda, "ada_boost")
print "#############====================== Gaussian NB ==========================#############"
pipeNB = Pipeline([('select', featSelector), ('NB', GaussianNB())])
pipeNB, _,_,_,_,_,_  = do_classify(pipeNB, {}, dftouse,lcols, u'results',1, reuse_split=reuse_split_new)
evaluate(pipeNB, "gaussian_nb")
print "#############====================== Gradient Boosting ====================#############"
pipeGB = Pipeline([('select', featSelector), ('GB', GradientBoostingClassifier())])
pipeGB, _,_,_,_,_,_  = do_classify(pipeGB, {"GB__n_estimators": [5,10,20,40],"GB__learning_rate": [0.1,0.5,1.0]}, dftouse,lcols, u'results',1, reuse_split=reuse_split_new)
evaluate(pipeGB, "gradient_boosting")

#############====================== Log Regression =====================#############
using reuse split
############# based on standard predict ################
Accuracy on training data: 0.68
Accuracy on validation data: 0.73
Accuracy on test data:     0.69
[[79 46]
 [24 80]]
########################################################
accuracy on validation set: 0.579
ROI "pred" buy-only: 6.09%
ROI "pred" buy-sell: 5.97%
accuracy on test set: 0.546
ROI "pred" buy-only: 7.34%
ROI "pred" buy-sell: 9.46%
#############====================== Linear SVM ========================#############
using reuse split
############# based on standard predict ################
Accuracy on training data: 0.69
Accuracy on validation data: 0.72
Accuracy on test data:     0.71
[[87 38]
 [28 76]]
########################################################
accuracy on validation set: 0.567
ROI "pred" buy-only: 5.98%
ROI "pred" buy-sell: 5.85%
accuracy on test set: 0.555
ROI "pred" buy-only: 12.58%
ROI "pred" buy-se

## Ensemble

In [47]:
predsv.keys()

['svm',
 'pipelinearsvm',
 'rbf_svc',
 'linear_svm',
 'ada_boost',
 'rand_forest',
 'pipesvm2',
 'extra_trees',
 'logistic_regression',
 'baseline_ema',
 'pipe_Log_Regr',
 'random_forest',
 'gaussian_nb',
 'gradient_boosting',
 'baseline_max',
 'baseline_long']

In [48]:
for k in predsv_ROI_long:
    print k + ' ROI buy-only: {0:.2f}%'.format(predsv_ROI_long[k]*100)

svm ROI buy-only: 6.93%
pipelinearsvm ROI buy-only: 5.01%
rbf_svc ROI buy-only: 6.46%
linear_svm ROI buy-only: 5.98%
ada_boost ROI buy-only: 4.62%
rand_forest ROI buy-only: 7.96%
pipesvm2 ROI buy-only: 6.43%
extra_trees ROI buy-only: 8.17%
logistic_regression ROI buy-only: 9.09%
baseline_ema ROI buy-only: 44.51%
pipe_Log_Regr ROI buy-only: 6.09%
random_forest ROI buy-only: 11.31%
gaussian_nb ROI buy-only: 13.26%
gradient_boosting ROI buy-only: 0.91%
baseline_max ROI buy-only: 133.45%
baseline_long ROI buy-only: 0.00%


In [49]:
for k in predsv_ROI_longshort:
    print k + ' ROI buy-only: {0:.2f}%'.format(predsv_ROI_longshort[k]*100)

svm ROI buy-only: 6.82%
pipelinearsvm ROI buy-only: 4.92%
rbf_svc ROI buy-only: 6.32%
linear_svm ROI buy-only: 5.85%
ada_boost ROI buy-only: 4.59%
rand_forest ROI buy-only: 7.81%
pipesvm2 ROI buy-only: 6.22%
extra_trees ROI buy-only: 8.02%
logistic_regression ROI buy-only: 8.98%
baseline_ema ROI buy-only: 45.61%
pipe_Log_Regr ROI buy-only: 5.97%
random_forest ROI buy-only: 11.72%
gaussian_nb ROI buy-only: 13.10%
gradient_boosting ROI buy-only: 0.76%
baseline_max ROI buy-only: 137.70%
baseline_long ROI buy-only: 0.00%


In [50]:
predst.keys()

['svm',
 'pipelinearsvm',
 'rbf_svc',
 'linear_svm',
 'ada_boost',
 'rand_forest',
 'pipesvm2',
 'extra_trees',
 'logistic_regression',
 'baseline_ema',
 'pipe_Log_Regr',
 'random_forest',
 'gaussian_nb',
 'gradient_boosting',
 'baseline_max',
 'baseline_long']

In [51]:
for k in predst_ROI_long:
    print k + ' ROI buy-only: {0:.2f}%'.format(predst_ROI_long[k]*100)

svm ROI buy-only: 5.44%
pipelinearsvm ROI buy-only: 6.54%
rbf_svc ROI buy-only: 15.58%
linear_svm ROI buy-only: 12.58%
ada_boost ROI buy-only: 6.22%
rand_forest ROI buy-only: 8.08%
pipesvm2 ROI buy-only: 15.31%
extra_trees ROI buy-only: 10.19%
logistic_regression ROI buy-only: 8.34%
baseline_ema ROI buy-only: 62.68%
pipe_Log_Regr ROI buy-only: 7.34%
random_forest ROI buy-only: -1.91%
gaussian_nb ROI buy-only: 8.08%
gradient_boosting ROI buy-only: 6.02%
baseline_max ROI buy-only: 153.57%
baseline_long ROI buy-only: 0.00%


In [52]:
for k in predst_ROI_longshort:
    print k + ' ROI buy-only: {0:.2f}%'.format(predst_ROI_longshort[k]*100)

svm ROI buy-only: 7.43%
pipelinearsvm ROI buy-only: 8.67%
rbf_svc ROI buy-only: 17.70%
linear_svm ROI buy-only: 14.90%
ada_boost ROI buy-only: 8.78%
rand_forest ROI buy-only: 10.23%
pipesvm2 ROI buy-only: 17.45%
extra_trees ROI buy-only: 12.39%
logistic_regression ROI buy-only: 10.47%
baseline_ema ROI buy-only: 68.01%
pipe_Log_Regr ROI buy-only: 9.46%
random_forest ROI buy-only: 0.37%
gaussian_nb ROI buy-only: 10.03%
gradient_boosting ROI buy-only: 8.12%
baseline_max ROI buy-only: 166.25%
baseline_long ROI buy-only: 0.00%


In [53]:
dfensemble=pd.DataFrame.from_dict({'svm':predsv['svm'],
                                     'pipelinearsvm':predsv['pipelinearsvm'],
                                     'rbf_svc':predsv['rbf_svc'],
                                     'linear_svm':predsv['linear_svm'],
                                     'ada_boost':predsv['ada_boost'],
                                     'rand_forest':predsv['rand_forest'],
                                     'pipesvm2':predsv['pipesvm2'],
                                     'extra_trees':predsv['extra_trees'],
                                     'logistic_regression':predsv['logistic_regression'],
                                     'pipe_Log_Regr':predsv['pipe_Log_Regr'],
                                     'random_forest':predsv['random_forest'],
                                     'gaussian_nb':predsv['gaussian_nb'],
                                     'gradient_boosting':predsv['gradient_boosting'],
                                   'y':predsv['baseline_ema']})

In [54]:
from sklearn.linear_model import LinearRegression
X = dfensemble.drop('y', axis = 1)
lm = LinearRegression()
valreg = lm.fit(X, dfensemble.y)


In [55]:
dfensembletest=pd.DataFrame.from_dict({'svm':predst['svm'],
                                     'pipelinearsvm':predst['pipelinearsvm'],
                                     'rbf_svc':predst['rbf_svc'],
                                     'linear_svm':predst['linear_svm'],
                                     'ada_boost':predst['ada_boost'],
                                     'rand_forest':predst['rand_forest'],
                                     'pipesvm2':predst['pipesvm2'],
                                     'extra_trees':predst['extra_trees'],
                                     'logistic_regression':predst['logistic_regression'],
                                     'pipe_Log_Regr':predst['pipe_Log_Regr'],
                                     'random_forest':predst['random_forest'],
                                     'gaussian_nb':predst['gaussian_nb'],
                                     'gradient_boosting':predst['gradient_boosting'],
                                   'y':predst['baseline_ema']})

In [56]:
Xt = dfensembletest.drop('y', axis = 1)
epreds = valreg.predict(Xt)

### Ensemble Results

In [57]:
#Test
df_pred = df[maskt].reset_index(drop=True)
df_pred['pred_result'] = 1*(epreds>0.5)
df_pred.head()

balance, profit, ROI, balovertime, signals = evaluate_profit(df_pred, sdatet, edatet, 10000, 'pred_result', 'close', False, [0])
print 'ROI "pred" buy-only: {0:.2f}%'.format(ROI*100)
#predst[desc] = ypred
#predst_ROI_long[desc] = ROI

balance, profit, ROI, balovertime, signals = evaluate_profit(df_pred, sdatet, edatet, 10000, 'pred_result', 'close', False, [1])
print 'ROI "pred" buy-sell: {0:.2f}%'.format(ROI*100)
#predst_ROI_longshort[desc] = ROI

ROI "pred" buy-only: 7.92%
ROI "pred" buy-sell: 10.03%
